<a href="https://colab.research.google.com/github/khalid-data-aspirant42/Used-Car-Price-Prediction/blob/main/Car_dekho_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [3]:
# Load your dataset
# Assuming your dataset is in a CSV file named 'car_data.csv'
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/AlmaBetter/Capstone Project/Project2:Supervised_Learning/Other projects/Car dekho dataset/Car_detailsv3.csv')

Mounted at /content/drive


In [5]:
data.head(3)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0


In [6]:
import numpy as np
def remove(df):
    df1 = df.replace('kmpl','')
    df2 = df1.replace('km/kg','')
    df3 = df2.replace('CC','')
    df4 = df3.replace('bhp','')
    df5 = df4.replace('nan','0')
    df6 = df5.replace(' ','')
    return df6

In [7]:
data['mileage'] = data['mileage'].astype('str').apply(remove)

data['engine'] = data['engine'].astype('str').apply(remove)

data['max_power'] = data['max_power'].astype('str').apply(remove)

In [8]:
# Preprocess the data
# Assuming 'Car_Name' is the column containing the names of the cars
car_names = data['name']

# Drop irrelevant columns (adjust as needed)
df = data.drop(['name', 'seller_type','owner','torque'], axis=1)

In [9]:
# Label encode categorical columns
le = LabelEncoder()
df['fuel'] = le.fit_transform(df['fuel'])

df['transmission'] = le.fit_transform(df['transmission'])

df['mileage'] = df['mileage'].astype('float')

df['engine'] = df['engine'].astype('int')

df['max_power'] = df['max_power'].replace('','0')

df['max_power'] = df['max_power'].astype('float')

df['seats'] = df['seats'].fillna(df['seats'].median())

In [10]:
df.isnull().sum()

df['fuel'].unique()

df.info()

df.corr()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           8128 non-null   int64  
 1   selling_price  8128 non-null   int64  
 2   km_driven      8128 non-null   int64  
 3   fuel           8128 non-null   int64  
 4   transmission   8128 non-null   int64  
 5   mileage        8128 non-null   float64
 6   engine         8128 non-null   int64  
 7   max_power      8128 non-null   float64
 8   seats          8128 non-null   float64
dtypes: float64(3), int64(6)
memory usage: 571.6 KB


,year,selling_price,km_driven,fuel,transmission,mileage,engine,max_power,seats
year,1.000000,0.414092,-0.418006,-0.050312,-0.244707,0.406930,0.129553,0.300136,0.010222
selling_price,0.414092,1.000000,-0.225534,-0.194029,-0.588845,-0.045405,0.445985,0.720341,0.047135
km_driven,-0.418006,-0.225534,1.000000,-0.265877,0.201229,-0.175519,0.155493,-0.060763,0.219860
fuel,-0.050312,-0.194029,-0.265877,1.000000,-0.035255,-0.075293,-0.440120,-0.262708,-0.337575
transmission,-0.244707,-0.588845,0.201229,-0.035255,1.000000,0.111906,-0.273145,-0.513393,0.069070
mileage,0.406930,-0.045405,-0.175519,-0.075293,0.111906,1.000000,-0.139442,-0.027941,-0.308898
engine,0.129553,0.445985,0.155493,-0.440120,-0.273145,-0.139442,1.000000,0.752893,0.580143
max_power,0.300136,0.720341,-0.060763,-0.262708,-0.513393,-0.027941,0.752893,1.000000,0.203256
seats,0.010222,0.047135,0.219860,-0.337575,0.069070,-0.308898,0.580143,0.203256,1.000000


In [11]:
df.describe()

,year,selling_price,km_driven,fuel,transmission,mileage,engine,max_power,seats
count,8128.000000,8.128000e+03,8.128000e+03,8128.000000,8128.000000,8128.000000,8128.000000,8128.000000,8128.000000
mean,2013.804011,6.382718e+05,6.981951e+04,1.891117,0.870817,18.890787,1418.965059,89.085848,5.405389
std,4.044249,8.062534e+05,5.655055e+04,1.002313,0.335423,5.082403,550.735342,38.286222,0.948874
min,1983.000000,2.999900e+04,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
25%,2011.000000,2.549990e+05,3.500000e+04,1.000000,1.000000,16.395000,1196.000000,67.100000,5.000000
50%,2015.000000,4.500000e+05,6.000000e+04,1.000000,1.000000,19.100000,1248.000000,81.860000,5.000000
75%,2017.000000,6.750000e+05,9.800000e+04,3.000000,1.000000,22.277500,1582.000000,101.250000,5.000000
max,2020.000000,1.000000e+07,2.360457e+06,3.000000,1.000000,42.000000,3604.000000,400.000000,14.000000


In [13]:
# Split the data into features (X) and target variable (y)
X = df.drop('selling_price', axis=1)
y = df['selling_price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Initialize and train a RandomForestRegressor (you can use other models as well)
rf = RandomForestRegressor()

final_rf_model = RandomForestRegressor()
model = final_rf_model.fit(X_train, y_train)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=200)

In [16]:
### Measuring the performance

y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)

from sklearn import metrics
# Computing scores
print(f'Train Score:{metrics.r2_score(y_train,y_pred_train)}')
print(f'Test Score:{metrics.r2_score(y_test,y_pred_test)}')

# Computing errors
print(f'MAE:{metrics.mean_absolute_error(y_test,y_pred_test)}')
print(f'MSE:{metrics.mean_squared_error(y_test,y_pred_test)}')
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred_test))
print("RMSE : % f" %(rmse))

print(f'Overall Model Score{model.score(X_test, y_test)}')

Train Score:0.9900576601884227
Test Score:0.9817628030944165
MAE:40337.59007614999
MSE:13094763052.695223
RMSE :  114432.351425
Overall Model Score0.9817628030944165


In [17]:
X_test[1000:1001]

,year,km_driven,fuel,transmission,mileage,engine,max_power,seats
6383,2019,100000,3,1,17.57,1193,88.7,5.0


In [18]:
y_test[1000:1001]

6383    620000
Name: selling_price, dtype: int64

In [19]:
a = int(input('year:'))
b = int(input('km_driven'))
# Getting input c (Fuel)
known_categories = ['diesel', 'petrol', 'lpg','cng']

# Example user input
user_input = input("Enter a category: ('Diesel', 'Petrol', 'LPG','CNG')").lower()

# Check if the user input is in the list of known categories
if user_input in known_categories:
    # Find the index of the user input in the list
    c = known_categories.index(user_input)

    print(f"Encoded input: {c}")
else:
    print("User input is not in the list of known categories.")


# Getting input d (Transmission)
known_categories = ['manual', 'automatic']

# Example user input
user_input = input("Enter a category: ('Manual', 'Automatic')").lower()

# Check if the user input is in the list of known categories
if user_input in known_categories:
    # Find the index of the user input in the list
    d = known_categories.index(user_input)

    print(f"Encoded input: {d}")
else:
    print("User input is not in the list of known categories.")


e = input('mileage')
f = input('engine')
g = input('max_power(hp):')
h = input('seats')

list1 = [a,b,c,d,e,f,g,h]
x = model.predict([list1])
print('Final Prediction:')
print(f'Selleing Price of the old car Rs. {x}')

int(x-50000)

print('List of 10 cars under the desired price range:')
data[(data['selling_price']>int(x-1000)) & (data['selling_price']<int(x+1000))].sort_values(by='year').tail(10)['name']

year:2019
km_driven100000
Enter a category: ('Diesel', 'Petrol', 'LPG','CNG')CNG
Encoded input: 3
Enter a category: ('Manual', 'Automatic')Automatic
Encoded input: 1
mileage17.57
engine1193
max_power(hp):88.7
seats5.0
Final Prediction:
Selleing Price of the old car Rs. [583099.96]
List of 10 cars under the desired price range:


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Series([], Name: name, dtype: object)

------------------------------------------------------------------------------------------------------------------------------------------